In [5]:
from ultralytics import YOLO
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import skimage
from PIL import Image

from sklearn.metrics import mean_squared_error

import json
import yaml
import time
import supervision as sv
from inference import get_model
print(sv.__version__)

0.25.0


In [6]:
src="./testvid.mp4"
target="./out.mp4"

In [8]:
import supervision as sv
from inference import get_model
import sink
from collections import deque

class BallTracker:
    def __init__(self, buffer_size: int = 10):
        self.buffer = deque(maxlen=buffer_size)

    def update(self, detections: sv.Detections) -> sv.Detections:
        xy = detections.get_anchors_coordinates(sv.Position.CENTER)
        self.buffer.append(xy)

        if len(detections) == 0:
            return detections

        centroid = np.mean(np.concatenate(self.buffer), axis=0)
        distances = np.linalg.norm(xy - centroid, axis=1)
        index = np.argmin(distances)
        return detections[[index]]

model = get_model(
    model_id="football-ball-detection-rejhg/4", 
    api_key="fpgo7lotAA2MTZARBCtt"
)

video_info = sv.VideoInfo.from_video_path(src)
frame_generator = sv.get_video_frames_generator(src)
w, h = video_info.width, video_info.height

def callback(patch: np.ndarray) -> sv.Detections: 
    result = model.infer(frame, confidence=0.3)[0]
    return sv.Detections.from_inference(result)

slicer = sv.InferenceSlicer(
    callback = callback,
    overlap_filter = sv.OverlapFilter.NON_MAX_SUPPRESSION,
    slice_wh = (w // 2 + 100, h // 2 + 100),
    overlap_ratio_wh = None,
    overlap_wh = (100, 100),
    iou_threshold = 0.1
)

annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FF1493'),
    height=20,
    base=25
)

tracker = BallTracker()
count=0
with sv.VideoSink(target, video_info=video_info) as sink:
    for frame in frame_generator:
        detections = slicer(frame)
        detections = tracker.update(detections)
        print('--detection--',detections)
        frame = annotator.annotate(scene=frame, detections=detections)
        sink.write_frame(frame)
        count+=1
        if count==10:
            break

--detection-- Detections(xyxy=array([[        325,         425,         338,         437]]), mask=None, confidence=array([    0.46147]), class_id=array([0]), tracker_id=None, data={'class_name': array(['ball'], dtype='<U4')}, metadata={})
--detection-- Detections(xyxy=array([[        334,         432,         347,         445]]), mask=None, confidence=array([      0.601]), class_id=array([0]), tracker_id=None, data={'class_name': array(['ball'], dtype='<U4')}, metadata={})
--detection-- Detections(xyxy=array([[        342,         438,         354,         451]]), mask=None, confidence=array([    0.77967]), class_id=array([0]), tracker_id=None, data={'class_name': array(['ball'], dtype='<U4')}, metadata={})
--detection-- Detections(xyxy=array([[        350,         445,         362,         458]]), mask=None, confidence=array([    0.71821]), class_id=array([0]), tracker_id=None, data={'class_name': array(['ball'], dtype='<U4')}, metadata={})
--detection-- Detections(xyxy=array([[      